In [ ]:
%matplotlib notebook

In [ ]:
from mocpy import TimeMOC
from astropy.time import Time, TimeDelta
from astroquery.vizier import Vizier

### Loading a TimeMoc from a fits file

In [ ]:
time_moc = TimeMOC.from_fits(
    "http://alasky.u-strasbg.fr/HST-hips/filter_SDSSr_hips/TMoc.fits"
)
time_moc.plot(title="HST_SDSSr tmoc")

### Loading a TimeMoc from an astropy table

In [ ]:
viz = Vizier(columns=["*", "_RAJ2000", "_DEJ2000"])
viz.ROW_LIMIT = -1
table = viz.get_catalogs("II/285")[1]

print(table)

In [ ]:
%%time
table_moc = TimeMOC.from_times(Time(table["JD"], format="jd", scale="tdb"))
table_moc.plot(title="2nd table of II/285 tmoc")

# print characteristics such as the time of the first/last observations
print("Time of the first observation:", table_moc.min_time.iso)
print("Time of the last observation:", table_moc.max_time.iso)
# the total duration of the observation times
print("Total duration: {0} jd".format(table_moc.total_duration.jd))
# the order of the TimeMoc
print("max order:", table_moc.max_order)

### Filtering an astropy table with a TimeMoc

In [ ]:
# filtering the table through the tmoc created from the HST_SDSSr fits file
rows = time_moc.contains_with_timeresolution(
    times=Time(table["JD"], format="jd", scale="tdb"),
    keep_inside=True,
    delta_t=TimeDelta(3600, format="sec", scale="tdb"),
)
print(table["JD"][rows])

### TimeMoc operation examples :
* #### Intersection of 2 tmocs : (HST_SDSSr) INTER (2nd table of II/285)

In [ ]:
result = table_moc.intersection_with_timeresolution(
    time_moc, delta_t=TimeMOC.order_to_time_resolution(9)
)

time_moc.plot(title="HST_SDSSr tmoc")
table_moc.plot(title="2nd table of II/285 tmoc")
result.plot(title="(HST_SDSSr) INTER (2nd table of II/285) tmoc")

# print the max order of all the tmocs. Result tmoc must be of order 9
print("HST_SDSSr max order : ", time_moc.max_order)
print("2nd table of II/285 max order : ", table_moc.max_order)
print("(HST_SDSSr) INTER (2nd table of II/285) max order : ", result.max_order)

* #### Augment and Diminish a tmoc

In [ ]:
result.add_neighbours()
result.plot(title="(HST_SDSSr) INTER (2nd table of II/285) augmented tmoc")

result.remove_neighbours()
result.plot(title="(HST_SDSSr) INTER (2nd table of II/285) diminished tmoc")

* #### Complement of a TimeMoc

In [ ]:
complemented_tmoc = result.complement()
complemented_tmoc.plot(
    title="Complement of (HST_SDSSr) INTER (2nd table of II/285)",
    view=(result.min_time, result.max_time),
)